In [2]:
!pip install google-generativeai

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 6.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 9.7 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.2 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
  Attempting uninstall: cachetools90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  6/23 [protobuf]
    Found existing installation: cach

In [7]:
# Cell 2 (Corrected)

import pandas as pd
import google.generativeai as genai
import json
import os

API_KEY = os.getenv("GOOGLE_API_KEY")

if API_KEY:
    genai.configure(api_key=API_KEY)
    
    # --- THIS IS THE CORRECTED LINE ---
    model = genai.GenerativeModel('gemini-1.5-flash-latest') 
    
    print("Gemini API configured successfully.")
else:
    print("API Key not found. Please run 'export GOOGLE_API_KEY=...' in your terminal before starting Jupyter.")
    model = None

Gemini API configured successfully.


In [8]:
# Load the reconciliation data and see its messy column names
recon_df = pd.read_csv('../data/reconciliation_challenge_data.csv')
print("--- Source Reconciliation Data Columns: ---")
print(recon_df.columns.tolist())
recon_df.head()

--- Source Reconciliation Data Columns: ---
['client_reference', 'full_customer_name', 'contact_email', 'transaction_ref', 'item_reference', 'transaction_date', 'amount_paid', 'payment_status', 'delivery_status', 'customer_segment', 'region', 'product_line', 'quantity_ordered', 'unit_cost', 'total_value', 'discount_applied', 'shipping_fee', 'tax_amount', 'notes_comments', 'last_modified_timestamp']


,client_reference,full_customer_name,contact_email,transaction_ref,item_reference,transaction_date,amount_paid,payment_status,delivery_status,customer_segment,region,product_line,quantity_ordered,unit_cost,total_value,discount_applied,shipping_fee,tax_amount,notes_comments,last_modified_timestamp
0,CLI_0404,Jane Doe,reconcile1@test.com,TXN_00865,ITM_159,10/28/2023,168.84,PENDING,IN_TRANSIT,STANDARD,EAST,BOOKS,10,223.96,771.53,36.64,17.15,30.76,NaN,2023-02-12T18:30:00.000Z
1,CLI_0295,John Smith,reconcile2@test.com,TXN_00146,ITM_160,12/13/2023,338.62,FAILED,IN_TRANSIT,VIP,EAST,ELECTRONICS,8,109.10,843.79,31.78,26.34,33.27,NaN,2023-10-22T18:30:00.000Z
2,CLI_0012,Bob Wilson,reconcile3@test.com,TXN_00175,ITM_004,10/9/2023,756.51,FAILED,IN_TRANSIT,PREMIUM,SOUTH,FASHION,5,86.28,509.81,40.80,6.38,18.40,NaN,2023-03-19T18:30:00.000Z
3,CLI_0049,Bob Wilson,reconcile4@test.com,TXN_00146,ITM_048,7/25/2023,886.05,FAILED,PENDING,STANDARD,EAST,BOOKS,10,107.95,556.04,81.54,10.96,9.19,NaN,2023-09-27T18:30:00.000Z
4,CLI_0267,John Smith,reconcile5@test.com,TXN_00245,ITM_009,3/16/2023,489.60,PENDING,IN_TRANSIT,VIP,NORTH,ELECTRONICS,8,231.53,308.35,89.58,23.53,62.68,NaN,2023-02-17T18:30:00.000Z


In [9]:
# This is the schema of our 'orders' table. This is what we want to map to.
target_schema = {
    'order_id': 'Unique identifier for the transaction.',
    'customer_id': 'The unique integer ID for the customer.',
    'product_id': 'The unique string ID for the product (e.g., ITM_159 or PROD_159).',
    'quantity': 'Number of units ordered.',
    'total_amount': 'The total monetary value of the order.',
    'order_date': 'The date the order was placed.',
    'status': 'The fulfillment or payment status of the order.'
}

In [10]:
# We give the AI a role, clear context, and strict instructions for the output format.
prompt = f"""
You are an expert data mapping assistant. Your task is to map columns from a new source CSV file to a target database schema.

**Target DB Schema (The one we need to match):**
{json.dumps(target_schema, indent=2)}

**Source CSV Columns (The new, messy file):**
{recon_df.columns.tolist()}

**Instructions:**
1.  Analyze the source column names to find the best semantic match for each target column.
2.  Create a JSON object where the keys are the **target schema columns** and the values are the corresponding **source column names**.
3.  If no clear match exists for a target column, use `null` as the value.
4.  Only output the final, valid JSON object and nothing else.
"""

schema_mapping = None
if model:
    print("--- Sending prompt to Gemini... ---")
    response = model.generate_content(prompt)
    try:
        # Clean up potential markdown formatting from the AI response
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '')
        schema_mapping = json.loads(cleaned_response)
        print("\n--- Schema Mapping Received from Gemini: ---")
        print(json.dumps(schema_mapping, indent=2))
    except Exception as e:
        print(f"\n--- Error parsing Gemini response: {e} ---")
        print("Response Text:", response.text)
else:
    print("Cannot run AI mapping without an API key.")

--- Sending prompt to Gemini... ---

--- Schema Mapping Received from Gemini: ---
{
  "order_id": "transaction_ref",
  "customer_id": null,
  "product_id": "item_reference",
  "quantity": "quantity_ordered",
  "total_amount": "total_value",
  "order_date": "transaction_date",
  "status": "payment_status"
}


In [11]:
if schema_mapping:
    # Create the renaming dictionary from the AI mapping: {source_name: target_name}
    rename_map = {v: k for k, v in schema_mapping.items() if v is not None}
    
    # Apply the renaming
    transformed_df = recon_df.rename(columns=rename_map)
    
    # Keep only the columns that are in our target schema
    final_columns = [col for col in target_schema.keys() if col in transformed_df.columns]
    final_df = transformed_df[final_columns]
    
    print("\n--- DataFrame after applying AI-generated mapping: ---")
    display(final_df.head())
else:
    print("\nCould not proceed with transformation.")


--- DataFrame after applying AI-generated mapping: ---


,order_id,product_id,quantity,total_amount,order_date,status
0,TXN_00865,ITM_159,10,771.53,10/28/2023,PENDING
1,TXN_00146,ITM_160,8,843.79,12/13/2023,FAILED
2,TXN_00175,ITM_004,5,509.81,10/9/2023,FAILED
3,TXN_00146,ITM_048,10,556.04,7/25/2023,FAILED
4,TXN_00245,ITM_009,8,308.35,3/16/2023,PENDING
